# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


## 0) Setup


In [1]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [2]:

corpus = [
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
]

# Train/test split at sentence level
random.seed(42) # Consistent output
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(10,
 4,
 ['printer driver install fails with error 1603',
  'push notifications not working on android app'],
 ['email delivery delayed messages queued',
  'vpn disconnects frequently after windows update'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [3]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [4]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 2
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['email', 'delivery', 'delayed', 'messages', 'queued'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>'])

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [5]:
def get_ngrams(tokens: list[str], n: int)-> list[tuple[str, ...]]:
   return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: list[str], n: int, vocab: set) -> Dict[tuple[str, ...], int]:
    ngram_counts = Counter()
    context_counts = Counter()
    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        for ng in get_ngrams(toks, n):
            ngram_counts[ng] += 1
            context = ng[:-1]
            context_counts[context] += 1
            
    return ngram_counts, context_counts


        


In [6]:
uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab)
uni_counts

Counter({('<UNK>',): 67,
         ('</s>',): 10,
         ('not',): 3,
         ('error',): 2,
         ('after',): 2})

In [7]:
bi_counts, bi_ctx = train_ngram_counts(train_texts, n=2, vocab=vocab)
bi_counts

Counter({('<UNK>', '<UNK>'): 51,
         ('<s>', '<UNK>'): 10,
         ('<UNK>', '</s>'): 10,
         ('<UNK>', 'not'): 3,
         ('not', '<UNK>'): 3,
         ('<UNK>', 'error'): 2,
         ('after', '<UNK>'): 2,
         ('error', '<UNK>'): 1,
         ('<UNK>', 'after'): 1,
         ('error', 'after'): 1})

In [8]:
tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab)
tri_counts

Counter({('<UNK>', '<UNK>', '<UNK>'): 38,
         ('<s>', '<s>', '<UNK>'): 10,
         ('<s>', '<UNK>', '<UNK>'): 10,
         ('<UNK>', '<UNK>', '</s>'): 7,
         ('<UNK>', '<UNK>', 'not'): 3,
         ('<UNK>', 'not', '<UNK>'): 3,
         ('<UNK>', '<UNK>', 'error'): 2,
         ('not', '<UNK>', '<UNK>'): 2,
         ('<UNK>', 'error', '<UNK>'): 1,
         ('error', '<UNK>', '</s>'): 1,
         ('not', '<UNK>', '</s>'): 1,
         ('<UNK>', '<UNK>', 'after'): 1,
         ('<UNK>', 'after', '<UNK>'): 1,
         ('after', '<UNK>', '<UNK>'): 1,
         ('<UNK>', 'error', 'after'): 1,
         ('error', 'after', '<UNK>'): 1,
         ('after', '<UNK>', '</s>'): 1})

## 5) Add-k smoothing and probability function


Add-k smoothing (a common baseline):
\na) Add *k* to every possible next word count  
b) Normalize by context_count + k * |V|

P_k(w|h) = (count(h,w) + k) / (count(h) + k*|V|)

Where V is the vocabulary.


In [9]:
def prob_addk(n_gram: tuple[str, ...], ngram_counts: Counter, context_counts: Counter, V: int, k: float = 0.5) -> float:
    """
    Compute add-k P(w_n | w_1 ...w_{n-1})
    where ngram = (w_1, w_2, ..., w_n)
    0<k<=1
    V is the vocabulary size
    """

    context = n_gram[:-1]
    return (ngram_counts[n_gram] + k) / (context_counts[context] + k * V)

In [10]:
V = len(vocab)
example = ("<s>", "login")
prob_addk(example, bi_counts, bi_ctx, V, k=0.5)


0.038461538461538464

## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [11]:

# Perplexity: matrix that tells if a model is better or bad. Lower perplexity is the better
# Evaluate perplexity on test set
def evaluate_perplexity(texts: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 0.5) -> float:
    V = len(vocab)
    log2_probs = []
    token_count = 0

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        ngrams = get_ngrams(toks, n)
        for ng in ngrams:
            p = prob_addk(ng, ngram_counts, context_counts, V, k=k)
            log2_probs.append(math.log(p, 2))
            token_count += 1

    H = -sum(log2_probs) / token_count
    PP = 2 ** H
    return PP

pp_uni = evaluate_perplexity(test_texts, n=1, ngram_counts=uni_counts, context_counts=uni_ctx, vocab=vocab, k=0.5)
pp_bi  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=0.5)
pp_tri = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=0.5)

pp_uni, pp_bi, pp_tri


(1.8224739937573902, 1.8712095221558307, 1.9552746520172761)

## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [12]:
def next_word_topk(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5, top_k: int = 5):
    """
    Given context tokens, return top-k next words with probabilities
    using add-k smoothing.
    """
        # Context length should be n-1
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    candidates = []
    for w in vocab:
        if w in {"<s>"}:
            continue
        ng = context + (w,)
        p = prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth)
        candidates.append((w, p))
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

# Bigram: context is 1 token
next_word_topk(["<s>"], n=2, ngram_counts=bi_counts, context_counts=bi_ctx, vocab=vocab, top_k=8)



[('<UNK>', 0.8076923076923077),
 ('not', 0.038461538461538464),
 ('error', 0.038461538461538464),
 ('after', 0.038461538461538464),
 ('</s>', 0.038461538461538464)]

In [13]:
next_word_topk(["<s>"], n=2, ngram_counts=bi_counts, context_counts=bi_ctx, vocab=vocab, k_smooth=0.5, top_k=5)

[('<UNK>', 0.8076923076923077),
 ('not', 0.038461538461538464),
 ('error', 0.038461538461538464),
 ('after', 0.038461538461538464),
 ('</s>', 0.038461538461538464)]

## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [14]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM:", generate(2, bi_counts, bi_ctx, vocab, max_len=18))


BIGRAM: not <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> not <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK>
BIGRAM: <UNK> <UNK> <UNK> <UNK> <UNK>
BIGRAM: <UNK>


## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [15]:
for k in [1.0, 0.5, 0.1, 0.01]:
    pp_bi_k  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=k)
    pp_tri_k = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=k)
    print(f"k={k:>4}:  bigram PP={pp_bi_k:,.2f}   trigram PP={pp_tri_k:,.2f}")



k= 1.0:  bigram PP=1.95   trigram PP=2.09
k= 0.5:  bigram PP=1.87   trigram PP=1.96
k= 0.1:  bigram PP=1.79   trigram PP=1.80
k=0.01:  bigram PP=1.76   trigram PP=1.75


## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen, fall back to unigram.  
4) Create a function that returns **top-5 next words** given a phrase like: `"user cannot"`.


### 1. Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.

In [24]:
more_sentences = [
    "vpn connection fails after router reboot",
    "user cannot login account locked out",
    "password reset email not received by user",
    "api returns 403 forbidden after token refresh",
    "portal page loads slow during peak hours",
    "email inbox not syncing on mobile device",
    "mfa code expires too quickly user complains",
    "wifi signal weak in conference room corner",
    "printer not detected after driver update",
    "teams video freezes when bandwidth drops",
    "shared drive access denied for new employee",
    "laptop battery not charging after bios update",
    "android app crashes when opening notifications",
    "mailbox storage quota exceeded cannot send email",
    "outlook keeps asking for password repeatedly",
    "network latency high after firewall change",
    "dns server not responding intermittently",
    "user session times out too fast on portal",
    "deployment caused database connection errors",
    "messages delayed in queue after service restart",
]

corpus2 = corpus + more_sentences
print("Old corpus size:", len(corpus))
print("New corpus size:", len(corpus2))


Old corpus size: 14
New corpus size: 34


In [25]:
random.seed(42)
random.shuffle(corpus2)
split = int(0.75 * len(corpus2))
train_texts = corpus2[:split]
test_texts  = corpus2[split:]

len(train_texts), len(test_texts)


(25, 9)

**Build Vocab**

In [26]:
# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 2
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)

(['wifi', 'signal', 'weak', 'in', 'conference', 'room', 'corner'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>'])

**Train n-gram Counts**

In [27]:
def get_ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: List[str], n: int, vocab: set) -> Tuple[Counter, Counter]:
    ngram_counts = Counter()
    context_counts = Counter()
    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        for ng in get_ngrams(toks, n):
            ngram_counts[ng] += 1
            context = ng[:-1]
            context_counts[context] += 1
    return ngram_counts, context_counts

uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab)
bi_counts, bi_ctx   = train_ngram_counts(train_texts, n=2, vocab=vocab)
tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab)

len(uni_counts), len(bi_counts), len(tri_counts)

(32, 91, 128)

**Evaluate cross entrpoy and perplexity**

In [28]:
def evaluate_perplexity(texts: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 0.5) -> float:
    V = len(vocab)
    log2_probs = []
    token_count = 0

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        ngrams = get_ngrams(toks, n)
        for ng in ngrams:
            p = prob_addk(ng, ngram_counts, context_counts, V, k=k)
            log2_probs.append(math.log(p, 2))
            token_count += 1

    H = -sum(log2_probs) / token_count
    PP = 2 ** H
    return PP

pp_uni = evaluate_perplexity(test_texts, n=1, ngram_counts=uni_counts, context_counts=uni_ctx, vocab=vocab, k=0.5)
pp_bi  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=0.5)
pp_tri = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=0.5)

pp_uni, pp_bi, pp_tri

(3.654585664417506, 4.430566853721905, 6.338462532560499)

**Obseveration**  
After adding 20 more sentences, the model usually gets better at predicting test text, so perplexity often decreases because it has seen more patterns.

### 2)  Change `min_count` (OOV threshold) and explain how perplexity changes. 

In [29]:
def build_vocab(train_texts, min_count):
    train_tokens_flat = []
    for t in train_texts:
        train_tokens_flat.extend(tokenize(t))
    freq = Counter(train_tokens_flat)

    vocab = {w for w, c in freq.items() if c >= min_count}
    vocab |= {"<UNK>", "<s>", "</s>"}
    return vocab

for mc in [1, 2, 3]:
    vocab_mc = build_vocab(train_texts, min_count=mc)

    uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab_mc)
    bi_counts, bi_ctx   = train_ngram_counts(train_texts, n=2, vocab=vocab_mc)
    tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab_mc)

    pp_uni = evaluate_perplexity(test_texts, n=1, ngram_counts=uni_counts, context_counts=uni_ctx, vocab=vocab_mc, k=0.5)
    pp_bi  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab_mc, k=0.5)
    pp_tri = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab_mc, k=0.5)

    print(f"min_count={mc} | Vocab size={len(vocab_mc)} | PP uni={pp_uni:.2f}, bi={pp_bi:.2f}, tri={pp_tri:.2f}")


min_count=1 | Vocab size=121 | PP uni=194.10, bi=122.04, tri=122.63
min_count=2 | Vocab size=33 | PP uni=3.65, bi=4.43, tri=6.34
min_count=3 | Vocab size=12 | PP uni=2.43, bi=2.54, tri=2.75


**Explanations**:  
When min_count =1 , Vocab size = 121
All words are kept, including many rare ones. The model sees many words only once, so it cannot predict them well in the test set. This leads to very high perplexity.  

When min_count =2 , Vocab size = 33
Rare words are mapped to UNK reducing sparsity. Model now can see more repeated patterns and hence can make better predictions with lower perplexity around 3-6.

When min_count =3 , Vocab size = 12
Most of the words are replaced by UNK. Hence model can see all most common tokens, resulting in very low perplexity, less surprising text and easy predictions.


### 3) Implement backoff (tri → bi → uni)

In [30]:
def prob_backoff(context_tokens, word,
                 uni_counts, uni_ctx,
                 bi_counts, bi_ctx,
                 tri_counts, tri_ctx,
                 vocab, k=0.5):
    V = len(vocab)

    # Try trigram if we have >=2 context tokens
    if len(context_tokens) >= 2:
        ng3 = (context_tokens[-2], context_tokens[-1], word)
        ctx3 = ng3[:-1]
        if tri_ctx[ctx3] > 0:
            return prob_addk(ng3, tri_counts, tri_ctx, V, k=k)

    # Try bigram if we have >=1 context token
    if len(context_tokens) >= 1:
        ng2 = (context_tokens[-1], word)
        ctx2 = ng2[:-1]
        if bi_ctx[ctx2] > 0:
            return prob_addk(ng2, bi_counts, bi_ctx, V, k=k)

    # Fall back to unigram
    ng1 = (word,)
    return prob_addk(ng1, uni_counts, uni_ctx, V, k=k)


In [31]:
def evaluate_perplexity_backoff(texts, vocab, k=0.5):
    log2_probs = []
    token_count = 0

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n=3)  # use trigram boundaries
        for i in range(2, len(toks)):
            context = toks[i-2:i]
            word = toks[i]
            p = prob_backoff(context, word, uni_counts, uni_ctx, bi_counts, bi_ctx, tri_counts, tri_ctx, vocab, k=k)
            log2_probs.append(math.log(p, 2))
            token_count += 1

    H = -sum(log2_probs) / token_count
    return 2 ** H

pp_backoff = evaluate_perplexity_backoff(test_texts, vocab=vocab, k=0.5)
print("Backoff perplexity:", pp_backoff)


Backoff perplexity: 3.6673595129423933


### 4) Create a function that returns **top-5 next words** given a phrase like: `"user cannot"`.

In [33]:
def next_word_top5_backoff(phrase: str, vocab: set, k=0.5, top_k=5):
    toks = replace_oov(tokenize(phrase), vocab)

    # Use trigram-style context: last 2 words, pad with <s> if needed
    context = ["<s>", "<s>"] + toks
    context = context[-2:]

    candidates = []
    for w in vocab:
        if w == "<s>":
            continue
        p = prob_backoff(context, w, uni_counts, uni_ctx, bi_counts, bi_ctx, tri_counts, tri_ctx, vocab, k=k)
        candidates.append((w, p))

    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

next_word_top5_backoff("user cannot", vocab=vocab, k=0.5, top_k=5)


[('login', 0.13513513513513514),
 ('for', 0.02702702702702703),
 ('outlook', 0.02702702702702703),
 ('on', 0.02702702702702703),
 ('out', 0.02702702702702703)]